In [1]:
# import opendatasets as od
# import os
import pandas as pd
import numpy as np
import copy
import itertools


In [10]:
base_rest_df = pd.read_csv('../restaurant_price_base.csv', sep=';')
base_rest_df.head()

,restaurant_link,country,region,province,city,avg_rating,total_reviews_count,price_level,price_min,price_max
0,g10001637-d10002227,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,4.0,36.0,€,14.0,29.0
1,g10001637-d14975787,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,4.0,5.0,€,8.0,17.0
2,g10002858-d4586832,France,Centre-Val de Loire,Berry,Rivarennes,5.0,13.0,€,10.0,35.0
3,g10002986-d3510044,France,Nouvelle-Aquitaine,Correze,Lacelle,4.0,34.0,€,12.0,26.0
4,g10022428-d9767191,France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,4.5,11.0,€€-€€€,12.0,30.0


In [11]:
base_rest_df.shape

(304327, 10)

In [12]:
base_rest_df.columns

Index(['restaurant_link', 'country', 'region', 'province', 'city',
       'avg_rating', 'total_reviews_count', 'price_level', 'price_min',
       'price_max'],
      dtype='object')

In [14]:
price_df = base_rest_df[['restaurant_link', 'country', 'region', 'province', 'city', 
                         'price_level', 
                         'avg_rating', 'total_reviews_count', 
                         'price_min', 'price_max']] #[base_rest_df['price_min'].notna()]

price_df.head()

,restaurant_link,country,region,province,city,price_level,avg_rating,total_reviews_count,price_min,price_max
0,g10001637-d10002227,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,€,4.0,36.0,14.0,29.0
1,g10001637-d14975787,France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,€,4.0,5.0,8.0,17.0
2,g10002858-d4586832,France,Centre-Val de Loire,Berry,Rivarennes,€,5.0,13.0,10.0,35.0
3,g10002986-d3510044,France,Nouvelle-Aquitaine,Correze,Lacelle,€,4.0,34.0,12.0,26.0
4,g10022428-d9767191,France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,€€-€€€,4.5,11.0,12.0,30.0


In [15]:
price_df.shape

(304327, 10)

In [20]:
price_df_short = price_df[['price_min', 'price_max']]

In [17]:
from pyod.models.iforest import IForest

In [21]:
iforest = IForest()

labels = iforest.fit_predict(price_df_short)
print(labels)

/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function fit_predict is deprecated
  warnings.warn(msg, category=FutureWarning)


[0 0 0 ... 0 0 0]


In [23]:
outliers = price_df_short[labels == 1]
print(outliers.shape)

(30429, 2)
